In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
# !pip install imblearn 

#### Import Relevant Libraries

In [2]:
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
import re, os
import string 
from nltk.tokenize import  word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD


import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import chi2, SelectKBest

#### Import Dataset

In [3]:
#import data
data_file = 'Amazon_Unlocked_Mobile.csv'
data_raw = pd.read_csv( data_file, delimiter = ",")

In [4]:
data=data_raw.copy()
data.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [5]:
data = data[data['Reviews'].isnull()==False]#remove any blank reviews


In [6]:
data['word_count'] = [len(review.split()) for review in data['Reviews']]


In [7]:
max_review=[data[data['word_count']== 5313].Reviews]
max_review

[131147    SUPPLEMENTAL USER GUIDE for BLU Pure XLBLU's o...
 131639    SUPPLEMENTAL USER GUIDE for BLU Pure XLBLU's o...
 Name: Reviews, dtype: object]

#### We can notice these two rows are showing user guide so we can remove them

In [8]:
data = data[data['word_count']<150]

In [9]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#### Data Preprocessing 

In [10]:
def review_to_wordlist( review, remove_stopwords=True):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()

    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (True by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    lemmatizer = WordNetLemmatizer()
    for word in words:
        b.append(lemmatizer.lemmatize(word))

    # 5. Return a list of words
    return(b)

#### Train test Split

In [11]:
X =data['Reviews']
y = data['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
clean_train = []
for review in X_train:
    clean_train.append( " ".join(review_to_wordlist(review)))
clean_val = []
for review in X_val:
    clean_val.append( " ".join(review_to_wordlist(review)))
    
clean_test = []
for review in X_test:
    clean_test.append( " ".join(review_to_wordlist(review)))

#### TF-IDF 

In [13]:
tfidfvec = TfidfVectorizer(ngram_range=(1, 3),max_features=200000,stop_words='english',token_pattern='(?u)\\b\\w\\w+\\b')

In [14]:

tfidfvec= tfidfvec.fit(clean_train)
clean_train_tf = tfidfvec.transform(clean_train)

clean_val_tf = tfidfvec.transform(clean_val)
clean_test_tf = tfidfvec.transform(clean_test)

#### Multinomial Naive Bayes Model

In [15]:
mnb=MultinomialNB(alpha=0.001)

#### SGD Classifier

In [18]:
sgd=SGDClassifier(loss='modified_huber', max_iter=5, random_state=0, shuffle=True)

#### Random Forest Classifier Model

In [15]:
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV

rfc=RandomForestClassifier(verbose =3,n_estimators=45)

#### AdaBoost Classifier

In [32]:
adc=AdaBoostClassifier(random_state=2)

#### XGBoost Classifier

In [21]:
xgb_model=XGBClassifier()

In [29]:
mnb.fit( clean_train_tf, y_train )

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [22]:
# pipeline2.fit( X_train, y_train )

In [30]:
sgd.fit( clean_train_tf, y_train )

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', max_iter=5,
       n_iter=None, n_jobs=1, penalty='l2', power_t=0.5, random_state=0,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [35]:
rfc.fit(clean_train_tf, y_train )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [37]:
adc.fit( clean_train_tf, y_train )

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [ ]:
xgb_model.fit(clean_train_tf, y_train )

In [42]:
preds1 = mnb.predict(clean_val_tf)#mnb

In [ ]:
# preds2 = pipeline1.predict(X_val)#svc

In [43]:
preds3 = sgd.predict(clean_val_tf)#sgd

In [44]:
preds4 = rfc.predict(clean_val_tf)#rfc

In [45]:
preds5 =adc.predict(clean_val_tf)#adaboost

In [46]:
preds6 = xgb_model.predict(clean_val_tf)#xgboost

/usr/local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [48]:
from  sklearn.metrics import classification_report,accuracy_score
print(classification_report(y_val, preds1))

             precision    recall  f1-score   support

          1       0.81      0.86      0.84     22806
          2       0.81      0.53      0.64      7427
          3       0.79      0.51      0.62      9796
          4       0.74      0.45      0.56     18638
          5       0.82      0.96      0.89     70849

avg / total       0.81      0.81      0.80    129516



In [ ]:
# print(classification_report(y_train, preds2))

In [49]:
print(classification_report(y_val, preds3))

             precision    recall  f1-score   support

          1       0.69      0.90      0.78     22806
          2       0.86      0.20      0.32      7427
          3       0.80      0.24      0.37      9796
          4       0.67      0.23      0.34     18638
          5       0.78      0.97      0.86     70849

avg / total       0.75      0.75      0.71    129516



In [50]:
print(classification_report(y_val, preds4))

             precision    recall  f1-score   support

          1       0.81      0.91      0.86     22806
          2       0.92      0.63      0.75      7427
          3       0.89      0.61      0.73      9796
          4       0.88      0.55      0.68     18638
          5       0.86      0.97      0.91     70849

avg / total       0.86      0.85      0.84    129516



In [51]:
print(classification_report(y_val, preds5))

             precision    recall  f1-score   support

          1       0.59      0.73      0.65     22806
          2       0.23      0.01      0.03      7427
          3       0.37      0.08      0.14      9796
          4       0.39      0.09      0.15     18638
          5       0.71      0.94      0.81     70849

avg / total       0.59      0.66      0.59    129516



In [52]:
print(classification_report(y_val, preds6))

             precision    recall  f1-score   support

          1       0.68      0.78      0.73     22806
          2       0.79      0.16      0.27      7427
          3       0.70      0.20      0.32      9796
          4       0.58      0.19      0.28     18638
          5       0.74      0.97      0.84     70849

avg / total       0.71      0.72      0.67    129516



In [54]:
print('model 1 accuracy: ', accuracy_score(y_val, preds1))
# print('model2 accuracy: ', accuracy_score(y_val, preds2))
print('model 3 accuracy: ', accuracy_score(y_val, preds3))
print('model 4 accuracy: ', accuracy_score(y_val, preds4))
print('model 5 accuracy: ', accuracy_score(y_val, preds5))
print('model 6 accuracy: ', accuracy_score(y_val, preds6))

model 1 accuracy:  0.812494209209673
model 3 accuracy:  0.7536134531640878
model 4 accuracy:  0.8534852836715154
model 5 accuracy:  0.6616093764476976
model 6 accuracy:  0.7182510269001513


#### SMOTE

In [17]:
from imblearn.over_sampling import SMOTE


Using TensorFlow backend.


In [18]:
X_train_resampled, y_train_resampled = SMOTE(random_state = 2).fit_sample(clean_train_tf, y_train)

In [25]:
mnb.fit( X_train_resampled, y_train_resampled )

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [26]:
sgd.fit( X_train_resampled, y_train_resampled)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=5, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
rfc.fit(X_train_resampled, y_train_resampled)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 45


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


building tree 2 of 45


In [33]:
adc.fit( X_train_resampled, y_train_resampled)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=2)

In [34]:
xgb_model.fit( X_train_resampled, y_train_resampled)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [35]:
preds1_smote = mnb.predict(clean_val_tf)#mnb

In [36]:
preds3_smote =sgd.predict(clean_val_tf)#sgd

In [21]:
preds4_smote = rfc.predict(clean_val_tf)#rfc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   12.1s finished


In [38]:
preds5_smote = adc.predict(clean_val_tf)#adc

In [39]:
preds6_smote = xgb_model.predict(clean_val_tf)#xgb

In [42]:
print(classification_report(y_val, preds1_smote))#mnb

              precision    recall  f1-score   support

           1       0.80      0.84      0.82     22806
           2       0.62      0.63      0.62      7427
           3       0.61      0.60      0.60      9796
           4       0.51      0.62      0.56     18638
           5       0.89      0.82      0.85     70849

    accuracy                           0.77    129516
   macro avg       0.68      0.70      0.69    129516
weighted avg       0.78      0.77      0.77    129516



In [43]:
print(classification_report(y_val, preds3_smote))#sgd

              precision    recall  f1-score   support

           1       0.75      0.83      0.79     22806
           2       0.43      0.56      0.48      7427
           3       0.49      0.50      0.50      9796
           4       0.50      0.47      0.48     18638
           5       0.87      0.83      0.85     70849

    accuracy                           0.74    129516
   macro avg       0.61      0.64      0.62    129516
weighted avg       0.74      0.74      0.74    129516



In [22]:
print(classification_report(y_val, preds4_smote))#RFC

              precision    recall  f1-score   support

           1       0.81      0.93      0.87     22806
           2       0.85      0.66      0.74      7427
           3       0.75      0.66      0.70      9796
           4       0.62      0.68      0.65     18638
           5       0.90      0.88      0.89     70849

    accuracy                           0.83    129516
   macro avg       0.79      0.76      0.77    129516
weighted avg       0.83      0.83      0.83    129516



In [23]:
print('model 4 rfc accuracy with SMOTE: ',accuracy_score(y_val, preds4_smote))

model 4 rfc accuracy with SMOTE:  0.8290944748139226


In [45]:
print(classification_report(y_val, preds5_smote))#ADABoost

              precision    recall  f1-score   support

           1       0.44      0.73      0.55     22806
           2       0.17      0.19      0.18      7427
           3       0.21      0.17      0.19      9796
           4       0.27      0.34      0.30     18638
           5       0.83      0.62      0.71     70849

    accuracy                           0.54    129516
   macro avg       0.38      0.41      0.38    129516
weighted avg       0.60      0.54      0.55    129516



In [46]:
print(classification_report(y_val, preds6_smote))#XGB

              precision    recall  f1-score   support

           1       0.67      0.79      0.72     22806
           2       0.28      0.43      0.34      7427
           3       0.44      0.37      0.40      9796
           4       0.37      0.48      0.42     18638
           5       0.86      0.72      0.78     70849

    accuracy                           0.65    129516
   macro avg       0.52      0.56      0.53    129516
weighted avg       0.69      0.65      0.67    129516



In [47]:
print('model 1(mnb classifier) accuracy with SMOTE: ', accuracy_score(y_val, preds1_smote))
# print('model2 accuracy: ', metrics.accuracy_score(y_val, preds2))
print('model 3sgd accuracy with SMOTE: ', accuracy_score(y_val, preds3_smote))
print('model 4 rfc accuracy with SMOTE: ',accuracy_score(y_val, preds4_smote))
print('model 5 adaboost accuracy with SMOTE: ', accuracy_score(y_val, preds5_smote))
print('model 6  XGB accuracy with SMOTE: ', accuracy_score(y_val, preds6_smote))

model 1(sgd classifier) accuracy with SMOTE:  0.7684842027239878
model 3 accuracy with SMOTE:  0.7358241452793477
model 4 accuracy with SMOTE:  0.8240063003798759
model 5 accuracy with SMOTE:  0.5367676580499706
model 6 accuracy with SMOTE:  0.652668396182711


#### Feature Selection

In [24]:
fselect = SelectKBest(chi2 , k=10000)
train_features = fselect.fit_transform(clean_train_tf,y_train )
val_features= fselect.transform(clean_val_tf)
test_features = fselect.transform(clean_test_tf)

In [53]:
mnb.fit( train_features, y_train )

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [ ]:
svc.fit( train_features, y_train )

In [54]:
sgd.fit( train_features, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=5, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [55]:
rfc.fit( train_features, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 20


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.1s remaining:    0.0s


building tree 2 of 20


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   22.4s remaining:    0.0s


building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  3.8min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=3, warm_start=False)

In [56]:
adc.fit( train_features, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=2)

In [58]:
xgb_model.fit( train_features, y_train )

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [59]:
preds1_featselect = mnb.predict(val_features)#mnb

In [61]:
# preds2_featselect = svc.predict(val_features)# svc

In [62]:
preds3_featselect = sgd.predict(val_features)#SGD classifier


In [63]:
preds4_featselect = rfc.predict(val_features)#random forest

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    2.4s finished


In [64]:
preds5_featselect = adc.predict(val_features)#adaboost

In [65]:
preds6_featselect = xgb_model.predict(val_features)#xgb

In [66]:
print(classification_report(y_val, preds1_featselect))

              precision    recall  f1-score   support

           1       0.68      0.76      0.72     22806
           2       0.83      0.19      0.31      7427
           3       0.77      0.17      0.28      9796
           4       0.54      0.11      0.18     18638
           5       0.72      0.98      0.83     70849

    accuracy                           0.71    129516
   macro avg       0.71      0.44      0.46    129516
weighted avg       0.70      0.71      0.64    129516



In [67]:
print(classification_report(y_val, preds3_featselect))

              precision    recall  f1-score   support

           1       0.65      0.83      0.73     22806
           2       0.82      0.11      0.19      7427
           3       0.71      0.12      0.21      9796
           4       0.48      0.11      0.18     18638
           5       0.73      0.97      0.84     70849

    accuracy                           0.71    129516
   macro avg       0.68      0.43      0.43    129516
weighted avg       0.69      0.71      0.64    129516



In [68]:
print(classification_report(y_val, preds4_featselect))

              precision    recall  f1-score   support

           1       0.83      0.91      0.87     22806
           2       0.92      0.64      0.76      7427
           3       0.90      0.62      0.73      9796
           4       0.85      0.56      0.68     18638
           5       0.86      0.97      0.91     70849

    accuracy                           0.86    129516
   macro avg       0.87      0.74      0.79    129516
weighted avg       0.86      0.86      0.85    129516



In [69]:
print(classification_report(y_val, preds5_featselect))

              precision    recall  f1-score   support

           1       0.53      0.68      0.60     22806
           2       0.24      0.00      0.01      7427
           3       0.40      0.07      0.11      9796
           4       0.34      0.05      0.08     18638
           5       0.68      0.93      0.79     70849

    accuracy                           0.64    129516
   macro avg       0.44      0.35      0.32    129516
weighted avg       0.56      0.64      0.56    129516



In [70]:
print(classification_report(y_val, preds6_featselect))

              precision    recall  f1-score   support

           1       0.68      0.78      0.73     22806
           2       0.81      0.16      0.26      7427
           3       0.69      0.20      0.31      9796
           4       0.58      0.19      0.28     18638
           5       0.74      0.97      0.84     70849

    accuracy                           0.72    129516
   macro avg       0.70      0.46      0.48    129516
weighted avg       0.70      0.72      0.66    129516



In [71]:
print('model 1(mnb classifier) accuracy with feature selection: ', accuracy_score(y_val, preds1_featselect))
# print('model2 accuracy: ', metrics.accuracy_score(y_val, preds2))
print('model 3sgd accuracy with feature selection: ', accuracy_score(y_val, preds3_featselect))
print('model 4 rfc accuracy with feature selection: ',accuracy_score(y_val, preds4_featselect))
print('model 5 adaboost accuracy with feature selection: ', accuracy_score(y_val, preds5_featselect))
print('model 6  XGB accuracy with feature selection: ', accuracy_score(y_val, preds6_featselect))

model 1(mnb classifier) accuracy with feature selection:  0.708298588591371
model 3sgd accuracy with feature selection:  0.7065845146545601
model 4 rfc accuracy with feature selection:  0.8563111893511226
model 5 adaboost accuracy with feature selection:  0.6397356311189352
model 6  XGB accuracy with feature selection:  0.7170002161895055


#### We can conclude Random Forest Model is performing best in term of metrics. It will be our main model and we will perform gride search to further improve its performance.

#### Feature Selection with SMOTE

In [25]:
fselect = SelectKBest(chi2 , k=10000)
train_features = fselect.fit_transform(X_train_resampled, y_train_resampled)
val_features= fselect.transform(clean_val_tf)
test_features = fselect.transform(clean_test_tf)

#### Grid Search CV with cross validation

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
bootstrap = [False]
max_depth = [30, 40, 50]
max_features = ['sqrt']
min_samples_leaf = [1, 2, 4]
min_samples_split = [5, 10, 15]
n_estimators = [100]

param_grid = {
    'bootstrap': bootstrap,
    'max_depth': max_depth,
    'max_features': max_features,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'n_estimators': n_estimators
}

# Create a base model
rfc = RandomForestClassifier(random_state=8)

# Manually create the splits in CV in order to be able to fix a random_state (GridSearchCV doesn't have that argument)
cv_sets = ShuffleSplit(n_splits = 3, test_size = .33, random_state = 8)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rfc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv_sets,
                           verbose=1)

# Fit the grid search to the data
grid_search.fit(train_features, y_train_resampled)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


#### best parameters turn out to be

In [ ]:
print("The best hyperparameters from Grid Search are:")
print(grid_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(grid_search.best_score_)

In [ ]:
best_rfc = grid_search.best_estimator_

#### Model fit and performance

In [ ]:
best_rfc.fit(train_features, y_train_resampled)

In [ ]:
rfc_pred = best_rfc.predict(test_features)

#### Training accuracy

In [ ]:
# Training accuracy
print("The training accuracy is: ")
print(accuracy_score(y_train_resampled, best_rfc.predict(train_features)))

#### Test Accuracy

In [ ]:
# Test accuracy
print("The test accuracy is: ")
print(accuracy_score(y_test, rfc_pred))

#### Classification Report